In [1]:
import pandas as pd

### Data Explanation 

1. The first column shows time and date range (from march of 2017 until september of 2017). 
  
  
2. Some columns were sampled every 20 second. Others were sampled on a hourly base.


3. The second and third columns are quality measures of the iron ore pulp right before it is fed into the flotation plant. 


4. Column 4 until column 8 are the most important variables that impact in the ore quality in the end of the process. 


5. From column 9 until column 22, we can see process data (level and air flow inside the flotation columns, which also impact in ore quality. 


6. The last two columns are the final iron ore pulp quality measurement from the lab.


7. Target is to predict the last column, which is the % of silica in the iron ore concentrate.


General comments:

1. Column % Iron Concentrate is correlated to % Silica Concentrate and represents dataleakage. 


2. However past samplings of % Silica and % Iron Concentrate are not leakage as they inform process operations and past information that can influence current yield


3. The above represents a regression problem 


In [2]:
df = pd.read_csv('../../data/raw/miningprocess_flotation_plant_database.csv.zip')

### Exploratory Data Analysis

In [ ]:
# Columns 

# 'date' ------------------------------------> Need to check sampling rate changes accross time and investigate

# '% Iron Feed' -----------------------------> Need to understand relationship wrt to % Silica Feed

# '% Silica Feed'---------------------------->

# 'Starch Flow'------------------------------>

# 'Amina Flow'------------------------------->

# 'Ore Pulp Flow'---------------------------->

# 'Ore Pulp pH'------------------------------> Need to treat as log scale potentially + worry about noise on instrument if needed

# 'Ore Pulp Density'-------------------------> Potentially inaccurate measurement 

# 'Flotation Column 02 Air Flow'------------>

# 'Flotation Column 03 Air Flow'------------>

# 'Flotation Column 04 Air Flow'------------>

# 'Flotation Column 05 Air Flow'------------>

# 'Flotation Column 06 Air Flow'------------>

# 'Flotation Column 07 Air Flow'------------> How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 01 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 02 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 03 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 04 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 05 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 06 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# 'Flotation Column 07 Level'---------------->How are these correlated? Are these at the same level or measured at different points

# '% Iron Concentrate'-----------------------> Correlated with % Silica Concentrate

# '% Silica Concentrate'---------------------> Final target column

### Datetime Analysis